In [7]:
pip install mysqlclient

   ---------------------------------------- 0.0/203.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/203.2 kB ? eta -:--:--
   ------- ------------------------------- 41.0/203.2 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 203.2/203.2 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pymysql
pymysql.install_as_MySQLdb()

In [13]:
pip install SQLObject pandas yfinance requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
Companies.deleteMany()  # 모든 Companies 데이터를 삭제

TypeError: You must give a where clause or pass in None to indicate no where clause

In [1]:
from sqlobject import SQLObject, sqlhub, connectionForURI
from sqlobject.col import StringCol, DecimalCol, DateCol
import pandas as pd
import yfinance as yf
import requests

# 데이터베이스 설정
db_user = 'root'
db_pass = '43qjsrnfjddl!@'
db_host = 'localhost'
db_name = 'sp500_db'
connection_string = f'mysql://{db_user}:{db_pass}@{db_host}/{db_name}'
sqlhub.processConnection = connectionForURI(connection_string)

# 테이블 정의 - 각 필드에 적절한 size, precision 및 scale 추가
class Companies(SQLObject):
    _idName = 'symbol'  # 기본 키로 사용될 열의 이름을 'symbol'로 설정
    symbol = StringCol(length=10, unique=True, alternateID=True)
    companyName = StringCol(length=255, dbName="companyName")
    industry = StringCol(length=255)
    sector = StringCol(length=255)

class StockPrices(SQLObject):
    symbol = StringCol(length=10)
    name = StringCol(length=255)
    sector = StringCol(length=255)
    country = StringCol(length=255)

class NewsArticles(SQLObject):
    symbol = StringCol(length=10)
    company = StringCol(length=255)
    title = StringCol(varchar=False)  # 긴 텍스트를 위해 올바르게 조정됨
    date = DateCol()
    url = StringCol(length=255)

class EnterpriseData(SQLObject):
    symbol = StringCol(length=10)
    companyName = StringCol(length=255)
    industry = StringCol(length=255)
    sector = StringCol(length=255)
    netIncomeToCommon = DecimalCol(size=10, precision=2)
    totalRevenue = DecimalCol(size=10, precision=2)
    ebitda = DecimalCol(size=10, precision=2)
    totalDebt = DecimalCol(size=10, precision=2)
    currentRatio = DecimalCol(size=10, precision=2)
    returnOnAssets = DecimalCol(size=10, precision=2)
    returnOnEquity = DecimalCol(size=10, precision=2)
    grossMargins = DecimalCol(size=10, precision=2)
    operatingMargins = DecimalCol(size=10, precision=2)

# 테이블 생성 코드 유지
Companies.createTable(ifNotExists=True)
StockPrices.createTable(ifNotExists=True)
NewsArticles.createTable(ifNotExists=True)
EnterpriseData.createTable(ifNotExists=True)

def save_companies_data():
    sp500 = pd.read_csv('./SP500_Companies.csv')
    for index, row in sp500.iterrows():
        stock = yf.Ticker(row['Symbol'])
        info = stock.info
        Companies(symbol=info.get("symbol"), companyName=info.get("longName"),
                   industry=info.get("industry"), sector=info.get("sector"))

def save_stock_prices():
    sp500 = pd.read_csv('./SP500_Companies.csv')
    for index, row in sp500.iterrows():
        stock = yf.Ticker(row['Symbol'])
        info = stock.info
        StockPrices(symbol=row['Symbol'], name=info.get('shortName'),
                     sector=info.get('sector'), country=info.get('country'))

def fetch_news(api_key, symbol, companyName, from_date, to_date, language='en'):
    query = f'"{symbol}" OR "{companyName}"'
    url = 'https://newsapi.org/v2/everything'
    params = {'q': query, 'from': from_date, 'to': to_date, 'language': language, 'sortBy': 'publishedAt', 'apiKey': api_key}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        articles = response.json()['articles']
        for article in articles:
            NewsArticles(symbol=symbol, company=companyName, title=article['title'],
                         date=article['publishedAt'][:10], url=article['url'])

def save_all_news():
    api_key = 'aa385e54196f4d278833fd4f8b663b1e'
    sp500 = pd.read_csv('./SP500_Companies.csv')
    for index, row in sp500.iterrows():
        fetch_news(api_key, row['Symbol'], row['Company'], '2024-02-01', '2024-02-29')

def save_enterprise_data():
    sp500 = pd.read_csv('./SP500_Companies.csv')
    for index, row in sp500.iterrows():
        stock = yf.Ticker(row['Symbol'])
        info = stock.info
        EnterpriseData(symbol=info.get("symbol"), companyName=info.get("longName"),
                       industry=info.get("industry"), sector=info.get("sector"),
                       netIncomeToCommon=info.get("netIncomeToCommon", 0),
                       totalRevenue=info.get("totalRevenue", 0),
                       ebitda=info.get("ebitda", 0), totalDebt=info.get("totalDebt", 0),
                       currentRatio=info.get("currentRatio", 0.0),
                       returnOnAssets=info.get("returnOnAssets", 0.0),
                       returnOnEquity=info.get("returnOnEquity", 0.0),
                       grossMargins=info.get("grossMargins", 0.0),
                       operatingMargins=info.get("operatingMargins", 0.0))

def update_or_insert_company(symbol, companyName, industry, sector):
    company = Companies.selectBy(symbol=symbol).getOne(None)
    if company:
        company.set(companyName=companyName, industry=industry, sector=sector)
    else:
        Companies(symbol=symbol, companyName=companyName,
                  industry=industry, sector=sector)
# 함수 호출 유지
save_companies_data()
save_stock_prices()
save_all_news()
save_enterprise_data()


DuplicateEntryError: Duplicate entry 'MSFT' for key 'companies.PRIMARY'